Inspired from
1. https://www.kaggle.com/chumajin/brain-tumor-eda-for-starter-english-version
2. https://www.kaggle.com/furcifer/torch-efficientnet3d-for-mri-no-train

In [ ]:
import numpy as np
import pandas as pd
import os
import pydicom
import matplotlib.pyplot as plt
from pydicom.pixel_data_handlers.util import apply_voi_lut
import glob
import cv2
import torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader
from torch import optim

import warnings
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
np.seterr(divide='ignore', invalid='ignore')

In [ ]:
labels = pd.read_csv("../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv")
labels.head()

In [ ]:
labels.MGMT_value.value_counts()

In [ ]:
def load_dicom(path , voi_lut=True, fix_monochrome=True , image_size = 256):
    dicom = pydicom.read_file(path)
    # VOI LUT (if available by DICOM device) is used to
    # transform raw DICOM data to "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    data = cv2.resize(data, (image_size, image_size))
    return data

In [ ]:
#min depth should be 20 , delete the files which have less than depth 20
def load_3d(path , image_size = 256 , depth =32):
    flair = sorted(glob.glob(f"{path}/FLAIR/*.dcm"))
    t1w = sorted(glob.glob(f"{path}/T1w/*.dcm"))
    t1wce = sorted(glob.glob(f"{path}/T1wCE/*.dcm"))
    t2w = sorted(glob.glob(f"{path}/T2w/*.dcm"))
    
    depth_per_source = depth // 4
    
    flair_img = np.array([load_dicom(a , image_size) for a in flair[len(flair)//2 - depth_per_source//2:len(flair)//2 + depth_per_source//2]]).T
    if flair_img.shape[-1] < depth_per_source:
        n_zero = depth_per_source - flair_img.shape[-1]
        flair_img = np.concatenate((flair_img, np.zeros((image_size, image_size, n_zero))), axis = -1)
    
    t1w_img = np.array([load_dicom(a , image_size) for a in t1w[len(t1w)//2 - depth_per_source//2:len(t1w)//2 + depth_per_source//2]]).T
    if t1w_img.shape[-1] < depth_per_source:
        n_zero = depth_per_source - t1w_img.shape[-1]
        t1w_img = np.concatenate((t1w_img, np.zeros((image_size, image_size, n_zero))), axis = -1)
    
    t1wce_img = np.array([load_dicom(a , image_size) for a in t1wce[len(t1wce)//2 - depth_per_source//2:len(t1wce)//2 + depth_per_source//2]]).T
    
    if t1wce_img.shape[-1] < depth_per_source:
        n_zero = depth_per_source - t1wce_img.shape[-1]
        t1wce_img = np.concatenate((t1wce_img, np.zeros((image_size, image_size, n_zero))), axis = -1)
  
    t2w_img = np.array([load_dicom(a , image_size) for a in t2w[len(t2w)//2 - depth_per_source//2:len(t2w)//2 + depth_per_source//2]]).T
    if t2w_img.shape[-1] < depth_per_source:
        n_zero = depth_per_source - t2w_img.shape[-1]
        t2w_img = np.concatenate((t2w_img, np.zeros((image_size, image_size, n_zero))), axis = -1)
        
    
    
    image_3d =  np.concatenate((flair_img, t1w_img, t1wce_img, t2w_img), axis = -1) #shpe= height x width x depth
    image_3d = torch.tensor(image_3d)
    image_3d = image_3d.permute(2 , 1 ,0)
    
    return image_3d
    

In [ ]:
l = load_3d("../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00121")
l.shape #depth , width , height

In [ ]:
plt.figure(figsize=(12, 12))
for i in range(32): 
    plt.subplot(4, 8, i+1)
    plt.imshow(l[i])
    plt.axis("off")
    

In [ ]:
labels["imfolder"] = ['{0:05d}'.format(s) for s in labels["BraTS21ID"]]
path = "../input/rsna-miccai-brain-tumor-radiogenomic-classification/train"
labels['path'] = [os.path.join(path ,f) for f in labels["imfolder"]]
train = labels

In [ ]:
train.head()

In [ ]:
#torch dataset
class Brain(Dataset):
    def __init__(self , df , image_size = 256 , depth =32):
        self.df = df
        self.img_size =image_size
        self.d = depth
    
    def __len__(self):
        return(len(self.df))
    
    def __getitem__(self , idx):
        img_path = self.df.loc[idx,'path']
        img_3d = load_3d(img_path , self.img_size , self.d)
        img_3d = img_3d.unsqueeze(0) # channel_length , deth , width , height
        
        target = self.df.loc[idx,'MGMT_value']
        
        return img_3d , torch.tensor(target)
        

In [ ]:
data = Brain(train)
loader = DataLoader(data,shuffle=True,batch_size=2)
i ,t = next(iter(loader))

In [ ]:
i.shape , t

# Now the data is ready , to be fed through a 3D CNN model, this is just a base idea, you can add many things like how you will handle the differnt sources, augmentations , image size etc etc